<a href="https://colab.research.google.com/github/Az-Ks/AirQo-Ugandan-Air-Quality-Forecast/blob/master/new_data_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

In [0]:

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
train=pd.read_csv("../input/airqo-ugandan-air-quality-forecast-challenge-zindi/Train (1).csv")
test=pd.read_csv("../input/airqo-ugandan-air-quality-forecast-challenge-zindi/Test (1).csv")

In [0]:
train.head()

In [0]:
# covert features  fron string to List of values 
def replace_nan(x):
    if x=="nan":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]


In [0]:
train1 = train.copy()
test1 = test.copy()

In [0]:
# covert features  fron string to List of values 
def replace_nan1(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train1[feature]=train1[feature].apply(lambda x: [ replace_nan1(X) for X in x.replace("nan"," ").split(",")])
    test1[feature]=test1[feature].apply(lambda x: [ replace_nan1(X)  for X in x.replace("nan"," ").split(",")]) 

### Features engineering part

In [0]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [0]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    x["p1_"+col_name]=x[col_name].apply(percentile(1))
    x["p5_"+col_name]=x[col_name].apply(percentile(5))
    x["p10_"+col_name]=x[col_name].apply(percentile(10))
    x["p20_"+col_name]=x[col_name].apply(percentile(20))
    x["p30_"+col_name]=x[col_name].apply(percentile(30))
    x["p40_"+col_name]=x[col_name].apply(percentile(40))
    x["p60_"+col_name]=x[col_name].apply(percentile(60))
    x["p70_"+col_name]=x[col_name].apply(percentile(70))
    x["p80_"+col_name]=x[col_name].apply(percentile(80))
    x["p90_"+col_name]=x[col_name].apply(percentile(90))
    x["p95_"+col_name]=x[col_name].apply(percentile(95))
    x["p99_"+col_name]=x[col_name].apply(percentile(99))
    return x  
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]


In [0]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)
data2=pd.concat([train1,test1],sort=False).reset_index(drop=True)
data2.columns.tolist()

In [0]:
for col_name in tqdm(features):
    data2[col_name]=data2[col_name].apply(remove_nan_values)

In [0]:
for col_name in tqdm(features):
    data2=aggregate_features(data2,col_name)

In [0]:
data2.head(2)

In [0]:
data.shape,data2.shape

In [0]:
#['temp_'+str(i) for i in temp_df.columns]

In [0]:
data1 = pd.DataFrame()
for col_name in tqdm(features):
    temp_df = data[col_name].apply(lambda x: pd.Series(x.split(',')))
    temp_df.columns = [col_name +str(i) for i in temp_df.columns]
    data1 = pd.concat([data1,temp_df],axis=1)

In [0]:
for col_name in tqdm(data1.columns):
    data1[col_name] = data1[col_name].apply(replace_nan)

In [0]:
data1

In [0]:
for col_name in tqdm(data1.columns):
    data1[col_name] = data1[col_name].fillna(0)

In [0]:
data1.head()

In [0]:
data11 = data1.copy()
data11['ID'] =  data['ID']

In [0]:
data11.head()

In [0]:
def fun(x):
    if x.split('_')[0] == 'atmos':
        return x

In [0]:
atmos = list(filter(fun,data11.columns))

In [0]:
data11 = data11[['ID']+atmos]

In [0]:
data11_atmos = pd.melt(data11,id_vars=["ID"],var_name='atmos', value_name='value_atmos')


In [0]:
for i in tqdm(range(1, 15)):
    data11_atmos[f'magic_{i}'] = data11_atmos.sort_values(by='ID')['value_atmos'].shift(i).expanding().mean().fillna(method='ffill').sort_index()
    data11_atmos[f'magic2_{i}'] = data11_atmos.sort_values(by='ID')['value_atmos'].shift(-i).expanding().mean().fillna(method='bfill').sort_index()

In [0]:
data11_atmos[data11_atmos.ID == 'ID_train_1'].head(3)

In [0]:
data11_piv = pd.pivot_table(data11_atmos,index='ID',columns = 'atmos',values = 'magic_1')
data11_piv.columns = ['atmos_magic_1'+i for i in data11_piv.columns]
for i in tqdm(range(2,15)):
    temp = pd.pivot_table(data11_atmos,index='ID',columns = 'atmos',values = 'magic_'+str(i))
    temp.columns = ['atmos_magic_'+str(i)+j for j in temp.columns]
    data11_piv = pd.concat([data11_piv,temp],axis=1)

In [0]:
data11_piv.reset_index(inplace=True)

In [0]:
data11 = pd.merge(data11,data11_piv,on='ID',how='left')

In [0]:
m = data1.isnull().sum()
sum(m[m>0])

In [0]:
data11.head()

In [0]:
data2.drop(features,1,inplace=True)

In [0]:
data2

In [0]:
data1['ID'] = data['ID']
data1['target'] = data['target']
data1['location'] = data['location']

In [0]:
data1.head()

In [0]:
data1.shape

In [0]:
# data1 = pd.merge(data1,data11,on='ID',how='inner')
# data1.shape

In [0]:
train=data1[data1.target.notnull()].reset_index(drop=True)
test=data1[data1.target.isna()].reset_index(drop=True)

In [0]:
train1=data2[data2.target.notnull()].reset_index(drop=True)
test1=data2[data2.target.isna()].reset_index(drop=True)

In [0]:
del data,data1,data2
gc.collect()

In [0]:
train.head()

In [0]:
train1.drop(['ID','location','target'],axis=1,inplace=True)
test1.drop(['ID','location','target'],axis=1,inplace=True)

In [0]:
train.shape,test.shape

In [0]:
train = pd.concat([train,train1],axis=1)
test = pd.concat([test,test1],axis=1)
train.shape,test.shape

In [0]:
Experiment_name="simple_model"

In [0]:
train_id = train['ID']
test_id = test['ID']
y = train['target']

In [0]:
#train.drop(['ID','target'],axis=1,inplace=True)
#test.drop(['ID','target'],axis=1,inplace=True)

In [0]:
# from sklearn.preprocessing import LabelEncoder
# lab  = LabelEncoder()
# lab.fit(train["location"])

# train.location  = lab.transform(train.location)
# test.location  = lab.transform(test.location)

In [0]:
import category_encoders as ce
encoder = ce.CatBoostEncoder(cols=["location"])
encoder.fit(train, y)
train = encoder.transform(train)
test = encoder.transform(test)

In [0]:
train.shape,test.shape

In [0]:
X  = train.copy()

In [0]:
X['ID'] = train_id
test['ID'] = test_id
X['target'] = y

In [0]:
X.to_csv('train_df.csv',index=False)
test.to_csv('test_df.csv',index=False)

## **PART 2  // THIS PART WILL TAKE AROUND 2H++**

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split


from scipy.sparse import csr_matrix
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,BaggingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
#from ngboost import NGBRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,StratifiedKFold, GroupKFold
import datetime
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from fastai.tabular import *
import warnings
from tqdm import *
warnings.filterwarnings("ignore")
np.random.seed(0)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
# Memory reduction helper function:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns
        col_type = df[col].dtypes
        if col_type in numerics: #numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
train =X
test=test.drop('target',1)
train.shape,test.shape

In [0]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [0]:
temp = ["temp{}".format(i) for i in range(121)]
precip = ["precip{}".format(i) for i in range(121)]
rel_humidity = ["rel_humidity{}".format(i) for i in range(121)]
wind_dir = ["wind_dir{}".format(i) for i in range(121)]
wind_spd = ["wind_spd{}".format(i) for i in range(121)]
atmos_press = ["atmos_press{}".format(i) for i in range(121)]

In [0]:
rem_org = list(set(test.columns) - set(temp+precip+rel_humidity+wind_dir+wind_spd+atmos_press))
len(rem_org)

In [0]:
import re
def fun(x):
    return int(re.findall(r'\d+', x)[0])

In [0]:
l = ['precip','rel_humidity','wind_dir','wind_spd','atmos_press']

In [0]:
data  =  train[['ID']+temp]
data = data.melt(id_vars=["ID"],var_name="d", value_name='temp')
data['d'] = data['d'].apply(fun)
for n,i in enumerate([precip,rel_humidity,wind_dir,wind_spd,atmos_press]):
    data1  =  train[['ID']+i]
    data1 = data1.melt(id_vars=["ID"],var_name="d", value_name=l[n])
    data1['d'] = data1['d'].apply(fun)
    data = pd.merge(data,data1,on=['ID','d'],how='inner')
    print('*'*8,n)

In [0]:
data_t  =  test[['ID']+temp]
data_t = data_t.melt(id_vars=["ID"],var_name="d", value_name='temp')
data_t['d'] = data_t['d'].apply(fun)
for n,i in enumerate([precip,rel_humidity,wind_dir,wind_spd,atmos_press]):
    data1  =  test[['ID']+i]
    data1 = data1.melt(id_vars=["ID"],var_name="d", value_name=l[n])
    data1['d'] = data1['d'].apply(fun)
    data_t = pd.merge(data_t,data1,on=['ID','d'],how='inner')
    print('*'*8,n)

In [0]:
data['train'] = True
data_t['train'] = False

data = pd.concat([data,data_t])
data.reset_index(drop=True,inplace=True)
data['train'].value_counts()

In [0]:
data.head()

In [0]:
def roll1(h):
    if h <= 24:
        return('day1')
    elif (h>24) and (h<=48):
        return('day2')
    elif (h>48) and (h<=72):
        return('day3')
    elif (h>72) and (h<=96):
        return('day4')
    elif (h>96) and (h<=120):
        return('day5')
    
    
def roll2(h):
    if (h <= 6) or ((h-24)>0 and (h-24)<=6)or ((h-24*2)>0 and (h-24*2)<=6)or ((h-24*3)>0 and (h-24*3)<=6)or ((h-24*4)>0 and (h-24*4)<=6):
        return('Morning')
    elif (h <= 12) or ((h-24)>0 and (h-24)<=12)or ((h-24*2)>0 and (h-24*2)<=12)or ((h-24*3)>0 and (h-24*3)<=12)or ((h-24*4)>0 and (h-24*4)<=12):
        return('Noon')
    elif (h <= 18) or ((h-24)>0 and (h-24)<=18)or ((h-24*2)>0 and (h-24*2)<=18)or ((h-24*3)>0 and (h-24*3)<=18)or ((h-24*4)>0 and (h-24*4)<=18):
        return('Evening')
    else:
        return('Night')
    
def roll3(h):
    if (h <= 3) or ((h-24)>0 and (h-24)<=3)or ((h-24*2)>0 and (h-24*2)<=3)or ((h-24*3)>0 and (h-24*3)<=3)or ((h-24*4)>0 and (h-24*4)<=3):
        return('Mor1')
    elif (h <= 6) or ((h-24)>0 and (h-24)<=6)or ((h-24*2)>0 and (h-24*2)<=6)or ((h-24*3)>0 and (h-24*3)<=6)or ((h-24*4)>0 and (h-24*4)<=6):
        return('Mor2')
    elif (h <= 9) or ((h-24)>0 and (h-24)<=9)or ((h-24*2)>0 and (h-24*2)<=9)or ((h-24*3)>0 and (h-24*3)<=9)or ((h-24*4)>0 and (h-24*4)<=9):
        return('Noo1')
    elif (h <= 12) or ((h-24)>0 and (h-24)<=12)or ((h-24*2)>0 and (h-24*2)<=12)or ((h-24*3)>0 and (h-24*3)<=12)or ((h-24*4)>0 and (h-24*4)<=12):
        return('Noo2')
    elif (h <= 15) or ((h-24)>0 and (h-24)<=15)or ((h-24*2)>0 and (h-24*2)<=15)or ((h-24*3)>0 and (h-24*3)<=15)or ((h-24*4)>0 and (h-24*4)<=15):
        return('Eve1')
    elif (h <= 18) or ((h-24)>0 and (h-24)<=18)or ((h-24*2)>0 and (h-24*2)<=18)or ((h-24*3)>0 and (h-24*3)<=18)or ((h-24*4)>0 and (h-24*4)<=18):
        return('Eve2')
    elif (h <= 21) or ((h-24)>0 and (h-24)<=21)or ((h-24*2)>0 and (h-24*2)<=21)or ((h-24*3)>0 and (h-24*3)<=21)or ((h-24*4)>0 and (h-24*4)<=21):
        return('Nig1')
    else:
        return('Nig2')

def roll4(h):
    if (h <= 8) or ((h-24)>0 and (h-24)<=8)or ((h-24*2)>0 and (h-24*2)<=8)or ((h-24*3)>0 and (h-24*3)<=8)or ((h-24*4)>0 and (h-24*4)<=8):
        return('First')
    elif (h <= 16) or ((h-24)>0 and (h-24)<=16)or ((h-24*2)>0 and (h-24*2)<=16)or ((h-24*3)>0 and (h-24*3)<=16)or ((h-24*4)>0 and (h-24*4)<=16):
        return('Second')
    else:
        return('Third')
    
def roll5(h):
    if (h <= 12) or ((h-24)>0 and (h-24)<=12)or ((h-24*2)>0 and (h-24*2)<=12)or ((h-24*3)>0 and (h-24*3)<=12)or ((h-24*4)>0 and (h-24*4)<=12):
        return('F_half')
    else:
        return('S_half')

In [0]:
data['type_of_day3'] = data['d'].apply(roll5)
data['type_of_day2'] = data['d'].apply(roll4)
data['type_of_day1'] = data['d'].apply(roll3)
data['type_of_day'] = data['d'].apply(roll2)
data['day'] = data['d'].apply(roll1)

In [0]:
data.groupby(['train','day','type_of_day3']).count()['temp']

In [0]:
data[(data['train'] == True) & (data['day'] == 'day5') & (data['type_of_day'] == 'Morning')].groupby('d').count()['temp']

In [0]:
data = reduce_mem_usage(data)

In [0]:
def percentile(n) :
    def percentile_(x) : 
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [0]:
%%time
grp_data1 = data.drop(['d','train'],axis=1).groupby(['ID','day']).agg(['mean','max','min','std',percentile(1),percentile(5),percentile(10),percentile(20)
                                                              ,percentile(30),percentile(40),percentile(50),percentile(60),percentile(70),percentile(80)
                                                              ,percentile(90),percentile(95),percentile(99)])

grp_data1.columns = ['_'.join(col).strip() for col in grp_data1.columns.values]
grp_data1.reset_index(inplace = True)

In [0]:
grp_data1.head()

In [0]:
%%time
grp_data2 = data.drop(['d','train'],axis=1).groupby(['ID','type_of_day']).agg(['mean','max','min','std',percentile(1),percentile(5),percentile(10),percentile(20)
                                                              ,percentile(30),percentile(40),percentile(50),percentile(60),percentile(70),percentile(80)
                                                              ,percentile(90),percentile(95),percentile(99)])

grp_data2.columns = ['_'.join(col).strip() for col in grp_data2.columns.values]
grp_data2.reset_index(inplace=True)

In [0]:
grp_data2.head()

In [0]:
%%time
#data['day_type_of_day'] = data['day']+ '_' + data['type_of_day']


grp_data3 = data.drop(['d','train'],axis=1).groupby(['ID','type_of_day1']).agg(['mean','max','min','std',percentile(1),percentile(5),percentile(10),percentile(20)
                                                              ,percentile(30),percentile(40),percentile(50),percentile(60),percentile(70),percentile(80)
                                                              ,percentile(90),percentile(95),percentile(99)])

grp_data3.columns = ['_'.join(col).strip() for col in grp_data3.columns.values]
grp_data3.reset_index(inplace=True)

In [0]:
grp_data3.shape

In [0]:
%%time
grp_data4 = data.drop(['d','train'],axis=1).groupby(['ID','type_of_day2']).agg(['mean','max','min','std',percentile(1),percentile(5),percentile(10),percentile(20)
                                                              ,percentile(30),percentile(40),percentile(50),percentile(60),percentile(70),percentile(80)
                                                              ,percentile(90),percentile(95),percentile(99)])

grp_data4.columns = ['_'.join(col).strip() for col in grp_data4.columns.values]
grp_data4.reset_index(inplace=True)

In [0]:
grp_data4.shape

In [0]:
%%time
#data['day_type_of_day1'] = data['day']+ '_' + data['type_of_day1']


grp_data5 = data.drop(['d','train'],axis=1).groupby(['ID','type_of_day3']).agg(['mean','max','min','std',percentile(1),percentile(5),percentile(10),percentile(20)
                                                              ,percentile(30),percentile(40),percentile(50),percentile(60),percentile(70),percentile(80)
                                                              ,percentile(90),percentile(95),percentile(99)])

grp_data5.columns = ['_'.join(col).strip() for col in grp_data5.columns.values]
grp_data5.reset_index(inplace=True)

In [0]:
grp_data5.shape

In [0]:
grp_data5

In [0]:
grp_data11 = pd.pivot_table(data = grp_data1,index='ID',columns = 'day',values = 'temp_mean')
grp_data11.columns = ['temp_mean'+ i for i in grp_data11.columns]
grp_data11.reset_index(inplace=True)
for i in tqdm(grp_data1.drop(['ID','day','temp_mean'],axis=1).columns):
    temp = pd.pivot_table(data = grp_data1,index='ID',columns = 'day',values = i)
    temp.columns = [i+ j for j in temp.columns]
    temp.reset_index(inplace=True)
    grp_data11 = pd.merge(grp_data11,temp,on='ID',how='left')

In [0]:
grp_data11_col = list(grp_data11.columns)

In [0]:
grp_data21 = pd.pivot_table(data = grp_data2,index='ID',columns = 'type_of_day',values = 'temp_mean')
grp_data21.columns = ['temp_mean'+ i for i in grp_data21.columns]
grp_data21.reset_index(inplace=True)
for i in tqdm(grp_data2.drop(['ID','type_of_day','temp_mean'],axis=1).columns):
    temp = pd.pivot_table(data = grp_data2,index='ID',columns = 'type_of_day',values = i)
    temp.columns = [i+ j for j in temp.columns]
    temp.reset_index(inplace=True)
    grp_data21 = pd.merge(grp_data21,temp,on='ID',how='left')

In [0]:
grp_data21_col = list(grp_data21.columns)

In [0]:
grp_data31 = pd.pivot_table(data = grp_data3,index='ID',columns = 'type_of_day1',values = 'temp_mean')
grp_data31.columns = ['temp_mean'+ i for i in grp_data31.columns]
grp_data31.reset_index(inplace=True)
for i in tqdm(grp_data3.drop(['ID','type_of_day1','temp_mean'],axis=1).columns):
    temp = pd.pivot_table(data = grp_data3,index='ID',columns = 'type_of_day1',values = i)
    temp.columns = [i+ j for j in temp.columns]
    temp.reset_index(inplace=True)
    grp_data31 = pd.merge(grp_data31,temp,on='ID',how='left')

In [0]:
grp_data31_col = list(grp_data31.columns)

In [0]:
grp_data41 = pd.pivot_table(data = grp_data4,index='ID',columns = 'type_of_day2',values = 'temp_mean')
grp_data41.columns = ['temp_mean'+ i for i in grp_data41.columns]
grp_data41.reset_index(inplace=True)
for i in tqdm(grp_data4.drop(['ID','type_of_day2','temp_mean'],axis=1).columns):
    temp = pd.pivot_table(data = grp_data4,index='ID',columns = 'type_of_day2',values = i)
    temp.columns = [i+ j for j in temp.columns]
    temp.reset_index(inplace=True)
    grp_data41 = pd.merge(grp_data41,temp,on='ID',how='left')
    

In [0]:
grp_data41_col = list(grp_data41.columns)

In [0]:
grp_data51 = pd.pivot_table(data = grp_data5,index='ID',columns = 'type_of_day3',values = 'temp_mean')
grp_data51.columns = ['temp_mean'+ i for i in grp_data51.columns]
grp_data51.reset_index(inplace=True)
for i in tqdm(grp_data5.drop(['ID','type_of_day3','temp_mean'],axis=1).columns):
    temp = pd.pivot_table(data = grp_data5,index='ID',columns = 'type_of_day3',values = i)
    temp.columns = [i+ j for j in temp.columns]
    temp.reset_index(inplace=True)
    grp_data51 = pd.merge(grp_data51,temp,on='ID',how='left')
    

In [0]:
grp_data51_col = list(grp_data51.columns)

In [0]:
grp_data_all = pd.merge(grp_data11,grp_data21,on='ID')
grp_data_all = pd.merge(grp_data_all,grp_data31,on='ID')
grp_data_all = pd.merge(grp_data_all,grp_data41,on='ID')
grp_data_all = pd.merge(grp_data_all,grp_data51,on='ID')
grp_data_all.shape

In [0]:
train_df = pd.merge(train,grp_data_all,on='ID',how='left')
test_df = pd.merge(test,grp_data_all,on='ID',how='left')
train_df.shape,test_df.shape,train.shape,test.shape

In [0]:
train_df.to_csv('final_train_df.csv',index=False)
test_df.to_csv('final_test_df.csv',index=False)

In [0]:
pd.DataFrame(rem_org).to_csv('rem_org.csv',index=False)
pd.DataFrame(grp_data11_col).to_csv('grp_data11_col.csv',index=False)
pd.DataFrame(grp_data21_col).to_csv('grp_data21_col.csv',index=False)
pd.DataFrame(grp_data31_col).to_csv('grp_data31_col.csv',index=False)
pd.DataFrame(grp_data41_col).to_csv('grp_data41_col.csv',index=False)
pd.DataFrame(grp_data51_col).to_csv('grp_data51_col.csv',index=False)


In [0]:
train_df.shape,test_df.shape,train.shape,test.shape

((15539, 3087), (5035, 3086), (15539, 843), (5035, 842))